# Tracking the Properties of Real Packets

**TARDIS** has the functionality to track the properties of the *RPackets* that are generated when running the Simulation. The `rpacket_tracker` can track all the interactions a packet undergoes & thus keeps a track of the various properties, a packet may have.<br>Currently, the `rpacket_tracker` tracks the properties of all the rpackets in the *Last Iteration of the Simulation*. It generates a `List` that contains the individual instances of `RPacketCollection`{`Numba JITClass`}, for storing all the interaction properties as listed below.

<html>
    <body>
        The properties that are tracked are as follows :
        <ol>
            <li><b>index</b> - Index of the Packet</li>
            <li><b>seed</b> - Seed of the Packet</li>
            <li><b>status</b> - Current Status for the Packet Interaction</li>
            <li><b>r</b> - Radius of the Current Shell</li> 
            <li><b>nu</b> - Packet's Frequency</li> 
            <li><b>mu</b> - Propagation Direction of the Packet (cosine of the angle the packet’s path makes with the radial direction)</li>
            <li><b>energy</b> - Energy of the Packet</li>
            <li><b>shell_id</b> - Current Shell Id where the Packet is present</li>
        </ol>
    </body>
</html>

<div class="alert alert-warning">

Warning

Current implementation stores all the data for the interaction of the packets in a `list`, so it needs to accessed with a `list index` for each property for a particular `rpacket`. Examples for the same are shown as follows. 
</div>

## How to Setup the Tracking for the RPackets?

**TARDIS**' `rpacket_tracker` is configured via the `YAML` file. This functionality of tracking the packets is turned **off**, by default. This is due to that fact that using this property, may slow down the execution time for the Simulation. An example configuration can be seen below for setting up the *tracking*:

```yaml
... 
montecarlo:
...
tracking:
    track_rpacket: true
```

The `montecarlo` section of the **YAML** file now has a `tracking` sub section which holds the configuration properties for the `track_rpacket` & the `initial_array_length` (discussed later in the tutorial).

Let us see, the new `rpacket_tracker` in action.

In [14]:
from tardis.io.config_reader import Configuration

In [15]:
# Reading the Configuration stored in `tardis_config_packet_tracking.yml` into config

config = Configuration.from_yaml("tardis_example.yml")

In [16]:
# Checking the `tracking` section via the Schema

config["montecarlo"]["tracking"]

{'track_rpacket': False, 'initial_array_length': 10}

In [17]:
# Setting `r_packet_tracking` to True to turn on the Tracking 

config["montecarlo"]["tracking"]["track_rpacket"] = True

In [18]:
config["montecarlo"]["tracking"]

{'track_rpacket': True, 'initial_array_length': 10}

In [19]:
from tardis import run_tardis

In [20]:
# Running the simulation from the config

sim = run_tardis(config, show_convergence_plots=False, show_progress_bars=False)

[py.warnings         ][WARNING]  /home/dhruvs/Repositories/tardis/tardis/plasma/properties/radiative_properties.py:92: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)
 (warnings.py:110)


Now, the `tracked` properties can be accessed via the `rpacket_tracker` attribute of the `sim.runner` object. 

In [21]:
type(sim.runner.rpacket_tracker)

numba.typed.typedlist.List

It can be seen from the above code, that the `sim.runner.rpacket_tracker` is an instance of the `List` specifically *Numba Typed List*. The `RPacketCollection` class has the following structure for the properties : {More information in the **TARDIS API** for `RPacketCollection` class}

```python
# Basic structure for the RPacketCollection Class
class RPacketCollection:
   # Properties
    index
    seed
    status
    r
    nu
    mu
    energy
    shell_id
```

In [22]:
len(sim.runner.rpacket_tracker)

100000

To access these different properties, we may consider the following examples for the `rpacket_tracker`:
<br>In this Example, we are trying to access the properties of the packet at index `10`.<br>In a similar way, we can check for any property for any packet in the range of packets for the last iteration.

- Accessing the `index` property for the packet {`10`}:

In [23]:
sim.runner.rpacket_tracker[10].index

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10])

- Accessing the `seed` property for the packet {`10`}:

In [24]:
sim.runner.rpacket_tracker[10].seed

array([2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521, 2729103521, 2729103521,
       2729103521, 2729103521, 2729103521])

- Accessing the `status` property for the packet {`10`}:

In [25]:
sim.runner.rpacket_tracker[10].status

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1])

Thus, all other properties {`r`, `nu`, `mu`, `energy`, `shell_id`} can be accessed accordingly.

We can also see the total number of interactions of index `10` packet under went, with the following example:

In [27]:
len(sim.runner.rpacket_tracker[10].shell_id)

53

<div class="alert alert-warning">

Warning

If we  try to access `sim.runner.rpacket_tracker` property when we have the `track_rpacket` property in the `tracking` subsection of `montecarlo` config, turned off as follows `config["montecarlo"]["tracking"]["track_rpacket"] = False`, it will return `None`.  Error will be raised if we try to access the properties i.e. `seed`, `index`, etc.
</div>

<div class="alert alert-info">

Note
    
When we initialise the `RPacketCollection()` class, the properties arrays {`index`, `seed`, `status`, etc} are allocated certain length based on the `initial_array_length` parameter that can be set via the `initial_array_length` property under `montecarlo -> tracking` section of the configuration. The default size of the array is `10`. This variable is important as the number of interactions a packet may have is variable, thus we need to allocate space dynamically. This variable is used to compute the size and expand the array such that the properties are able to hold these values for the packet interaction. Higher number, allocates more space initially leading to lesser times the arrays expands and vice versa. It can be set in the following manner `config["montecarlo"]["tracking"]["initial_array_length"] = {value}`.
</div>